In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

import harness

np.warnings.filterwarnings('ignore', 'Ill-conditioned matrix')
pd.options.display.max_columns=None

In [2]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [3]:
flights_clean = harness.clean_train(flights_train)


add_features = harness.chain(
    harness.add_date_parts, 
    harness.add_haul, 
    harness.add_weather,
    harness.add_hour,
#    harness.make_all_dummies
)


flights_with_features = add_features(flights_clean)


x_transform = harness.chain(
    harness.keep_only_test_columns,
    add_features,
#    [harness.add_all_grouped_stats, flights_with_features],
    harness.only_numeric,
    harness.scale,
)


transformer = harness.DataTransformer(
    x_transform, harness.normalize, harness.unnormalize
)


x_train, y_train = transformer.extract_transform(flights_clean)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PowerTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [4]:
for col in x_train.columns:
    print(f"{col}: {np.corrcoef(x_train[col], y_train.arr_delay)[0, 1]}")

mkt_carrier_fl_num: 0.012862292432801372
op_carrier_fl_num: 0.01287439929672372
origin_airport_id: -0.009438814869731969
dest_airport_id: 0.0030847547005330104
crs_dep_time: 0.09771383315876599
crs_arr_time: 0.08795351101131096
crs_elapsed_time: -0.08631898665835405
distance: -0.06560419674987938
month: -0.00398405787624075
day: -0.0035219871001030594
hour: 0.0973902908650512


In [ ]:
def select_columns(df):
    return df[[
        'crs_dep_time',
        'crs_arr_time',
        'crs_elapsed_time',
        'distance',
        'origin_city_name_delay_mean',
        'dest_city_name_delay_mean',
        'origin_airport_id_delay_mean',
        'dest_airport_id_delay_mean',
        'day_delay_mean',
        'month_delay_mean',
        'op_unique_carrier_delay_mean',
        'haul_delay_mean',
        'tail_num_delay_mean',
        'hour_delay_mean',
    ]]





x_transform = harness.chain(x_transform, select_columns)
transformer = harness.DataTransformer(
    x_transform
)
x_train, y_train = transformer.extract_transform(flights_clean)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)



{'colsample_bytree': [0.1,0.3,0.5,0.7,0.9],
 'learning_rate': [0.1,0.3,0.5,0.7,0.9],
 'max_depth' : [None, 2,3,5], 'alpha': [1,5,10]}



xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [6]:
def train_model():
    xg_reg = xgb.XGBRegressor(objective ='reg:linear',n_estimators=10)
    grid_search = ms.GridSearchCV(
        xg_reg, {'colsample_bytree': [0.1,0.3,0.5,0.7,0.9],
                 'learning_rate': [0.1,0.3,0.5,0.7,0.9],
                 'max_depth' : [None, 2,3,5], 'alpha': [1,5,10]})
    grid_search.fit(x_train.values, y_train.values)
    harness.save(grid_search.best_estimator_, 'XGBoost_model')
    

In [7]:
train_model()

[21:34:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:26] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:26] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is no

In [20]:
xg_reg = harness.load('XGBoost_model')
xg_reg_model = harness.TrainedModel(xg_reg, transformer)

xg_reg_model.validate(harness.clean_train(flights_test))

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10'] ['mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance', 'month', 'day', 'hour']
expected f10, f1, f3, f2, f8, f0, f7, f6, f9, f5, f4 in input data
training data did not have the following fields: hour, distance, dest_airport_id, crs_elapsed_time, day, mkt_carrier_fl_num, crs_arr_time, origin_airport_id, op_carrier_fl_num, crs_dep_time, month

In [ ]:
GBR_model.submit(
    'data/test.csv', 'time_of_day_submission.csv', 'predicted_delay'
)